#### Setup

In [1]:
# standard imports 
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch import optim
from ipdb import set_trace
from datetime import datetime

# jupyter setup
%matplotlib inline
%load_ext autoreload
%autoreload 2

# own modules
from dataloader import CAL_Dataset
from net import get_model
from dataloader import get_data, get_mini_data, load_json, save_json
from train import fit, custom_loss, validate
from metrics import calc_metrics

# paths
data_path = './dataset/'

uncomment the cell below if you want your experiments to yield always the same results

In [ ]:
# manualSeed = 42

# np.random.seed(manualSeed)
# torch.manual_seed(manualSeed)

# # if you are using GPU
# torch.cuda.manual_seed(manualSeed)
# torch.cuda.manual_seed_all(manualSeed)

# torch.backends.cudnn.enabled = False 
# torch.backends.cudnn.benchmark = False
# torch.backends.cudnn.deterministic = True

#### Training

Initialize the model. Possible Values for the task block type: MLP, LSTM, GRU, TempConv

In [ ]:
params = {'name': 'tempconv', 'type_': 'TempConv', 'lr': 3e-4, 'n_h': 128, 'p':0.5, 'seq_len':5}

In [ ]:
save_json(params, f"models/{params['name']}")

In [ ]:
model, opt = get_model(params)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

get the data loader. get mini data gets only a subset of the training data, on which we can try if the model is able to overfit

In [ ]:
train_dl, valid_dl = get_data(data_path, model.params.seq_len, batch_size=16)
# train_dl, valid_dl = get_mini_data(data_path, model.params.seq_len, batch_size=16, l=4000)

Train the model. We automatically save the model with the lowest val_loss. If you want to continue the training and keep the loss history, just pass it as an additional argument as shown below.

In [ ]:
model, val_hist = fit(1, model, custom_loss, opt, train_dl, valid_dl)

In [ ]:
# model, val_hist = fit(1, model, custom_loss, opt, train_dl, valid_dl, val_hist=val_hist)

uncomment the following two cells if the feature extractor should also be trained

In [ ]:
# for name,param in model.named_parameters():
#     param.requires_grad = True
# opt = optim.Adam(model.parameters())

In [ ]:
# model, val_hist = fit(1, model, custom_loss, opt, train_dl, valid_dl)

In [ ]:
plt.plot(val_hist)

#### evalute the model

reload model

In [ ]:
name = 'gru'
params = load_json(f"models/{name}")

In [ ]:
model, _ = get_model(params)
model.load_state_dict(torch.load(f"./models/{name}.pth"));

In [ ]:
model.eval().to(device);
_, valid_dl = get_data(data_path, model.params.seq_len, batch_size=16)

run evaluation on full val set

In [ ]:
_, all_preds, all_labels = validate(model, valid_dl, custom_loss)

In [ ]:
calc_metrics(all_preds, all_labels)

#### plot results

In [ ]:
# for convience, we can pass an integer instead of the full string
int2key = {0: 'red_light', 1:'hazard_stop', 2:'speed_sign', 
           3:'relative_angle', 4: 'center_distance', 5: 'veh_distance'}

In [ ]:
def plot_preds(k, all_preds, all_labels, start=0, delta=1000):
    if isinstance(k, int): k = int2key[k]
    
    # get preds and labels
    class_labels = ['red_light', 'hazard_stop', 'speed_sign']
    pred = np.argmax(all_preds[k], axis=1) if k in class_labels else all_preds[k]
    label = all_labels[k][:, 1] if k in class_labels else all_labels[k]
    
    plt.plot(pred[start:start+delta], 'r--', label='Prediction', linewidth=2.0)
    plt.plot(label[start:start+delta], 'g', label='Ground Truth', linewidth=2.0)
    
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
plot_preds(5, all_preds, all_labels, start=0, delta=4000)

#### param search



In [2]:
from numpy.random import choice
np.random.seed()

In [3]:
params = {'name': 'tempconv', 'type_': 'TempConv', 'lr': 3e-4, 'n_h': 128, 'p':0.5, 'seq_len':5}

In [8]:
def get_random_NN_parameters():
    params = {}
    params['type_'] = choice(['MLP', 'GRU', 'LSTM', 'TempConv'])
    params['name'] = datetime.now().strftime("%Y_%m_%d_%H_%M")
    params['lr'] = np.random.uniform(1e-5, 1e-2)
    params['n_h'] = np.random.randint(5, 200)
    params['p'] = np.random.uniform(0.25, 0.75)
    params['seq_len'] = np.random.randint(1, 15)
    
    return params

In [9]:
while True:
    params = get_random_NN_parameters()    
    print('PARAMS: {}'.format(params))
    
    # instantiate the model
    model, opt = get_model(params)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    save_json(params, f"models/{params['name']}")
              
    # get the data loaders
    train_dl, valid_dl = get_data(data_path, model.params.seq_len, batch_size=16)
    
    # start the training
    model, val_hist = fit(5, model, custom_loss, opt, train_dl, valid_dl)
    for name,param in model.named_parameters():
        param.requires_grad = True
        opt = optim.Adam(model.parameters())
    model, val_hist = fit(5, model, custom_loss, opt, train_dl, valid_dl, val_hist=val_hist)

PARAMS: {'name': '2019_06_13_19_55', 'lr': 0.007221551882651735, 'n_h': 48, 'p': 0.5985699357015684, 'seq_len': 14, 'type_': 'MLP'}


FileNotFoundError: [Errno 2] File b'./dataset/annotations.csv' does not exist: b'./dataset/annotations.csv'